In [1]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from glob import glob 
import os
from utilsdf import read_tiles_nosample

/home/ljp238/miniconda3/envs/agluon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/*/*_core.parquet"
fparquet_list = glob(path)

In [4]:
df = pd.read_parquet(fparquet_list[0])

In [6]:
df.isna().sum()

negroaoidtm        81018001
multi_dtm_lidar     8524192
edem_w84                  0
cdem_dem                  0
edem_egm                  0
multi_s1x_band1           0
multi_s1x_band2           0
multi_s2_band1            0
multi_s2_band2            0
multi_s2_band3            0
dtype: int64

In [8]:
df.head() # where cdem and edem0 is equal to zero, also make edeme to zero @find out exactly where 

,negroaoidtm,multi_dtm_lidar,edem_w84,cdem_dem,edem_egm,multi_s1x_band1,multi_s1x_band2,multi_s2_band1,multi_s2_band2,multi_s2_band3
0,NaN,NaN,-8.909220,0.0,0.0,0.101358,0.068114,0.103014,0.116391,0.128022
1,NaN,NaN,-8.908882,0.0,0.0,0.101396,0.068180,0.103996,0.117849,0.129290
2,NaN,NaN,-8.908545,0.0,0.0,0.103076,0.068425,0.103668,0.117686,0.129413
3,NaN,NaN,-8.908208,0.0,0.0,0.104756,0.068670,0.103340,0.117522,0.129536
4,NaN,NaN,-8.907657,0.0,0.0,0.106301,0.067329,0.103559,0.117577,0.129275


In [ ]:


fcol = ['edem_w84','multi_s1_band2', 'multi_s2_band1', 'multi_s2_band2', 'multi_s2_band3']
ldem = 'multi_dtm_lidar'
pdem = 'negroaoidtm'
rcol = 'edem_w84'
ycol = 'zdif'

s1col =  ['multi_s1_band1','multi_s1_band2']
s2col = ['multi_s2_band1', 'multi_s2_band2', 'multi_s2_band3']


# print(s2tcol)
# acol = list(set(s1tcol + s2tcol))
# acol

In [10]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def save_parquet_chunk(df_chunk, file_name):
    """Função auxiliar para salvar um chunk em Parquet"""
    df_chunk.to_parquet(file_name, index=False)
    print(f"Saved {file_name} with {len(df_chunk)} rows.")

def save_chunks_as_parquet(df, chunk_size=100_000, output_prefix="chunk", num_workers=8):
    """Divide o DataFrame em chunks e salva os arquivos em paralelo"""
    num_chunks = (len(df) // chunk_size) + 1
    tasks = []

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        for i, start in enumerate(range(0, len(df), chunk_size)):
            df_chunk = df.iloc[start:start + chunk_size]
            file_name = f"{output_prefix}_{i}.parquet"
            tasks.append(executor.submit(save_parquet_chunk, df_chunk, file_name))

        # Espera todas as tarefas finalizarem
        for task in tasks:
            task.result()

def save_chunks_as_parquets(df, chunk_size=100_000, output_prefix="chunk"):
    num_chunks = (len(df) // chunk_size) + 1
    for i, start in enumerate(range(0, len(df), chunk_size)):
        df_chunk = df.iloc[start:start + chunk_size]
        df_chunk.to_parquet(f"{output_prefix}_{i}.parquet", index=False)
        print(f"Saved {output_prefix}_{i}.parquet with rows.")#{len(df_chunk)} 

In [4]:
ldar_mkd = [
'/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N09E105/N09E105_core.parquet',
 '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N09E106/N09E106_core.parquet',
 '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N10E104/N10E104_core.parquet',
 '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N10E105/N10E105_core.parquet',
 '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N10E106/N10E106_core.parquet',
]

ldar_tls = ['/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N13E103/N13E103_core.parquet']

ldar_rgn = [
'/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/S01W063/S01W063_core.parquet',
 '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/S01W064/S01W064_core.parquet',
 '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/S02W063/S02W063_core.parquet',
 '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/S02W064/S02W064_core.parquet']

In [12]:
#df = pd.read_parquet(ldar_tls[0])

In [ ]:
parq_outdir = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/PARQ_CLEAN_TRAIN"

In [ ]:
n=100_000
px = 30
for roi in ['mkd', 'tls', 'rng']:
    if roi == 'mkd':
        print(f'processing {roi}')
        df = read_tiles_nosample(fparquet_list,rcol,ldem)
        df = df.drop(pdem,axis=1)
        df[ycol] = df[rcol].subtract(df[ldem])
        s1tcol = [ldem,rcol]+s1col
        s2tcol = [ldem,rcol]+s2col
        pqt = f"{parq_outdir}/{roi}/clean_data_"
        print(df.columns.tolist())
        save_chunks_as_parquet(df, chunk_size=n, output_prefix=pqt, num_workers=px)
        
    elif roi == 'tls':
        print(f'processing {roi}')
        df = read_tiles_nosample(fparquet_list,rcol,ldem)
        df = df.drop(pdem,axis=1)
        df[ycol] = df[rcol].subtract(df[ldem])
        s1tcol = [ldem,rcol]+s1col
        s2tcol = [ldem,rcol]+s2col
        outdir = f"{parq_outdir}/{roi}/clean_data_"
        print(df.columns.tolist())
        save_chunks_as_parquet(df, chunk_size=n, output_prefix=pqt, num_workers=px)

    elif roi == 'rng':
        print(f'processing {roi}')
        df = read_tiles_nosample(fparquet_list,rcol,pdem)
        df = df.drop(ldem,axis=1)
        s1tcol = [pdem,rcol]+s1col
        s2tcol = [pdem,rcol]+s2col 
        outdir = f"{parq_outdir}/{roi}/clean_data_"
        df.rename(columns={pdem:ldem})
        save_chunks_as_parquet(df, chunk_size=n, output_prefix=pqt, num_workers=px)
        print(df.columns.tolist())

Encoding nulls from column edem_w84...
Dropping nulls from column 'edem_w84'...


In [15]:
df.shape

(236435487, 9)

In [ ]:

parql_fn = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/PARQ_CLEAN_TRAIN/ROIL/clean_data_"
#save_chunks_as_parquet(df, chunk_size=100_000, output_prefix=parql_fn)

In [16]:
save_chunks_as_parquet(df, chunk_size=100_000, output_prefix=parql_fn, num_workers=30)

,negroaoidtm,multi_dtm_lidar,edem_w84,multi_s1_band1,multi_s1_band2,multi_s2_band1,multi_s2_band2,multi_s2_band3,zdif
724,NaN,0.076340,-0.070413,-19.272154,-26.251677,0.099683,0.117876,0.127815,0.146753
725,NaN,0.076999,-1.166148,-19.356813,-25.942974,0.096351,0.116226,0.127280,1.243147
726,NaN,0.077737,-2.261883,-19.199352,-25.456274,0.093231,0.114643,0.126582,2.339620
727,NaN,0.078648,-2.935314,-16.990135,-23.461210,0.091912,0.113635,0.124503,3.013963
728,NaN,0.079386,-3.273816,-14.780917,-21.466145,0.090592,0.112627,0.122425,3.353203
...,...,...,...,...,...,...,...,...,...
460257102,NaN,11.671014,-5.320837,-9.176203,-15.561838,0.100506,0.109037,0.118499,16.991852
460257103,NaN,11.948798,-4.785427,-8.502990,-14.942888,0.097290,0.107175,0.116694,16.734224
460257104,NaN,12.045464,-4.563872,-7.971846,-14.443746,0.094598,0.105806,0.115370,16.609337
460257105,NaN,12.161258,-4.342318,-7.823746,-14.267628,0.093318,0.105764,0.115340,16.503576


In [8]:
d1 = df.sample(1000).copy()

In [10]:
from zdev import train_all_models
output_dir = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/MODELS/"

In [14]:
del train_all_models

In [13]:
train_all_models(d1, output_dir, num_rounds=1000, seed=42)

0:	learn: 3.4582972	test: 3.3911306	best: 3.3911306 (0)	total: 8.17ms	remaining: 8.16s
bestTest = 2.214194887
bestIteration = 69
Shrink model to first 70 iterations.
0:	learn: 2.7711008	test: 2.2024711	best: 2.2024711 (0)	total: 6.95ms	remaining: 6.94s
bestTest = 2.178650067
bestIteration = 8
Shrink model to first 9 iterations.
0:	learn: 3.4855090	test: 3.4039158	best: 3.4039158 (0)	total: 8.06ms	remaining: 8.05s
bestTest = 2.397910162
bestIteration = 32
Shrink model to first 33 iterations.
0:	learn: 2.7707485	test: 2.2025917	best: 2.2025917 (0)	total: 8.99ms	remaining: 8.98s
bestTest = 2.202591705
bestIteration = 0
Shrink model to first 1 iterations.


{'model_s1id': {'Seed': 42,
  'RMSE': 2.2141951259113997,
  'R2': 0.6185697283386025,
  'ModelPath': '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/MODELS/model_s1id_1000_model.cbm'},
 'model_s1di': {'Seed': 42,
  'RMSE': 2.178650130127179,
  'R2': 0.011206108440865337,
  'ModelPath': '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/MODELS/model_s1di_1000_model.cbm'},
 'model_s2id': {'Seed': 42,
  'RMSE': 2.3979102216789445,
  'R2': 0.5526481740435532,
  'ModelPath': '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/MODELS/model_s2id_1000_model.cbm'},
 'model_s2di': {'Seed': 42,
  'RMSE': 2.2025916712170175,
  'R2': -0.010645334204750778,
  'ModelPath': '/media/ljp238/12TBWolf/RSPROX/OUT_TILES/MODELS/model_s2di_1000_model.cbm'}}